# Dividend Aristocrats

In this notebook, I am going to generate a list of companies called Dividend Aristocrats. There are some requirements for companies to be selected in the group of Dividend Aristocrats, I will make the process simple by selecting companies in S&P 500 which have paid out dividends at least 25 years. I can add more requirements on top of the threshold of 25 years.

## Get Stock Symbols from S&P 500

In [24]:
import yfinance as yf
import pandas as pd
import datetime as dt

In [13]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [14]:
sp500 = data[0]
sp500 = sp500[['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Date first added']]
sp500.head(10)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Date first added
0,MMM,3M Company,Industrials,Industrial Conglomerates,1976-08-09
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,1964-03-31
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,2012-12-31
3,ABMD,Abiomed,Health Care,Health Care Equipment,2018-05-31
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,2011-07-06
5,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,2015-08-31
6,ADBE,Adobe Inc.,Information Technology,Application Software,1997-05-05
7,AMD,Advanced Micro Devices,Information Technology,Semiconductors,2017-03-20
8,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,2015-07-09
9,AES,AES Corp,Utilities,Independent Power Producers & Energy Traders,1998-10-02


In [16]:
sp500_symbols = sp500['Symbol'].values.tolist()
sp500_sectors = list(set(sp500['GICS Sector'].values.tolist()))

## Retrieve Dividend Data

In [18]:
#for symbol in sp500_symbols:
symbol = 'JNJ'
prices = yf.Ticker(symbol).history(period='max')
dividends = prices[prices['Dividends'] > 0]
dividends.head(20)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1962-02-13,0.00000,0.076148,0.075246,0.075426,734400,0.000579,0.0
1962-05-22,0.00000,0.064061,0.062613,0.062613,432000,0.000579,0.0
1962-08-21,0.00000,0.053023,0.052297,0.052297,129600,0.000579,0.0
1962-11-21,0.00000,0.050937,0.050117,0.050937,475200,0.000579,0.0
1963-02-19,0.00000,0.059301,0.059118,0.059301,172800,0.000810,0.0
1963-05-21,0.00000,0.066442,0.065340,0.065524,864000,0.000579,0.0
1963-08-20,0.00000,0.066994,0.066809,0.066994,86400,0.000579,0.0
1963-11-26,0.00000,0.075844,0.073076,0.074736,2419200,0.000579,0.0
1964-02-25,0.00000,0.085588,0.084014,0.084014,604800,0.001042,0.0


In [38]:
first_year = dividends.index[0].year
last_year = dt.datetime.today().year

annual_div = {}

for year in range(first_year, last_year):
    annual_div[year] = dividends[dividends.index.year ==  year]['Dividends'].sum()

annual_div

{1962: 0.002316,
 1963: 0.0025470000000000002,
 1964: 0.0027790000000000002,
 1965: 0.0033550000000000003,
 1966: 0.003819,
 1967: 0.004283,
 1968: 0.004515,
 1969: 0.005903,
 1970: 0.007084,
 1971: 0.008957,
 1972: 0.009312,
 1973: 0.010937,
 1974: 0.015105,
 1975: 0.017709,
 1976: 0.021874,
 1977: 0.029168,
 1978: 0.035416,
 1979: 0.041668,
 1980: 0.046354000000000006,
 1981: 0.053229,
 1982: 0.060625,
 1983: 0.06718899999999998,
 1984: 0.073438,
 1985: 0.079689,
 1986: 0.08593800000000001,
 1987: 0.10062499999999999,
 1988: 0.12,
 1989: 0.14,
 1990: 0.16375,
 1991: 0.1925,
 1992: 0.2225,
 1993: 0.2525,
 1994: 0.28250000000000003,
 1995: 0.32,
 1996: 0.36749999999999994,
 1997: 0.425,
 1998: 0.485,
 1999: 0.545,
 2000: 0.6200000000000001,
 2001: 0.7,
 2002: 0.7949999999999999,
 2003: 0.9249999999999999,
 2004: 1.0949999999999998,
 2005: 1.2750000000000001,
 2006: 1.455,
 2007: 1.62,
 2008: 1.795,
 2009: 1.93,
 2010: 2.1100000000000003,
 2011: 2.2499999999999996,
 2012: 2.4,
 2013: 2.

2021

In [29]:

first_year + 1
annual_div = 0
annual_df = pd.DataFrame()

output = []
        


1963